In [2]:
import os
from dotenv import load_dotenv
load_dotenv()
project_path = os.getenv("PROJECT_PATH")

In [3]:
%run "{project_path}\llm_custom_apps\common_code\common_func.ipynb"

13


In [5]:
def get_excel_col_header(file_path):
    """
    Traverses through all sheets in the Excel file and returns a dictionary.
    
    Each key represents a sheet name, and its value is another dictionary where
    each key is the Column Name (from the first row) and the value is another 
    dictionary with key 'ColumnID' and value as the Column ID (column letter, e.g., 'A', 'B', 'C').

    :param file_path: Path to the Excel file
    :return: Dictionary with sheet names and column headers
    """
    result = {}

    # Load the workbook
    workbook = openpyxl.load_workbook(file_path, data_only=True)

    # Iterate through each sheet in the workbook
    for sheet_name in workbook.sheetnames:
        sheet = workbook[sheet_name]
        sheet_data = {}

        # Get the first row (header) for each column
        for column_idx, cell in enumerate(sheet[1], 1):  # sheet[1] gets the first row
            col_letter = openpyxl.utils.get_column_letter(column_idx)
            sheet_data[cell.value] = {'ColumnID': col_letter}  # Nested dict with 'ColumnID'

        # Add sheet data to the result dictionary
        result[sheet_name] = sheet_data

    return result

# Example Usage
# file_path = "path_to_your_excel_file.xlsx"
# result = get_excel_col_header(file_path)
# print(result)


In [7]:
def process_excel_columns_xlwings(file_path, sheet_columns_dict):
    """
    Updates the sheet_columns_dict with an additional key 'ColumnValue' for each field,
    extracting the formula (if applicable) or the value from the second row of the specified column.

    :param file_path: Path to the Excel spreadsheet.
    :param sheet_columns_dict: Dictionary containing sheet and column information.
    :return: Updated dictionary with 'ColumnValue' for each field.
    """
    # Open the workbook in xlwings
    app = xw.App(visible=False)
    wb = app.books.open(file_path)
    
    try:
        # Iterate over each sheet and its column configurations
        for sheet_name, columns in sheet_columns_dict.items():
            # Ensure the sheet exists in the workbook
            if sheet_name not in [sheet.name for sheet in wb.sheets]:
                raise ValueError(f"Sheet '{sheet_name}' not found in the Excel file.")

            # Get the worksheet
            ws = wb.sheets[sheet_name]

            # Process each column
            for column_name, column_info in columns.items():
                # Extract the column ID
                column_id = column_info['ColumnID']
                
                # Get the cell in the second row for the specified column
                cell = ws.range(f"{column_id}2")

                # Extract formula if applicable, else get the value
                column_value = cell.formula if cell.formula != "" else cell.value

                # Add the 'ColumnValue' key to the column_info
                column_info['ColumnValue'] = column_value

        # Return the updated dictionary
        return sheet_columns_dict
    
    finally:
        # Close the workbook and quit the app
        wb.close()
        app.quit()
